In [26]:
import json
# Load JSON data
with open("data/home0001qa.json", "r") as file:
    qa_data = json.load(file)

Because we have a question / answer dataset with relatively short answers, we don't needwe don't need to use a splitter

In [27]:
from langchain.schema import Document
# Prepare documents for LangChain
documents = [
    Document(page_content=item["answer"], metadata={"question": item["question"]})
    for item in qa_data
]

documents[:5]

[Document(metadata={'question': 'Do i own my 0001 home outright?'}, page_content='When you buy a 0001 home, you own the title in the traditional way. If you need, we’ll help you find the right mortgage and can recommend real estate lawyers. You keep full legal ownership of your home, with the added benefit that you can spend time in other locations whenever you want.'),
 Document(metadata={'question': 'Does furniture come included?'}, page_content='Yes, each new home comes fully furnished and equipped so that you can move in easily with just your suitcase. Each interior is designed for flexibility and functionality and we work directly with designers from our community to source pieces often straight from their studios.'),
 Document(metadata={'question': 'Can i change the design of my home?'}, page_content="Legally you own your home and are free to do what you want with it. However, to maintain access to home0001's network in other locations, your home does need to meet our standards a

In [28]:
# turn into function
def prepare_qa_documents(file_path):
    with open(file_path, 'r') as f:
        qa_data = json.load(f)
    
    documents = [
        Document(
            page_content=item["answer"],
            metadata={"question": item["question"]}
        )
        for item in qa_data
    ]
    
    return documents

print(prepare_qa_documents("data/home0001qa.json")[:5])

[Document(metadata={'question': 'Do i own my 0001 home outright?'}, page_content='When you buy a 0001 home, you own the title in the traditional way. If you need, we’ll help you find the right mortgage and can recommend real estate lawyers. You keep full legal ownership of your home, with the added benefit that you can spend time in other locations whenever you want.'), Document(metadata={'question': 'Does furniture come included?'}, page_content='Yes, each new home comes fully furnished and equipped so that you can move in easily with just your suitcase. Each interior is designed for flexibility and functionality and we work directly with designers from our community to source pieces often straight from their studios.'), Document(metadata={'question': 'Can i change the design of my home?'}, page_content="Legally you own your home and are free to do what you want with it. However, to maintain access to home0001's network in other locations, your home does need to meet our standards and

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [35]:
def setup_rag(documents):
    
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(documents, embeddings)
    
    llm = ChatOpenAI(model="gpt-4o-mini")
    prompt = ChatPromptTemplate.from_messages([
        ("system", """Use the following similar Q&A pairs to help answer the question. 
        If the context is relevant, use it to answer. If not, say you don't have enough information.
        
        Context Q&A pairs:
        {context}
        """),
        ("human", "{question}")
    ])
    
    chain = (
        {"context": vectorstore.as_retriever(search_type="similarity", k=3), "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return chain

In [36]:
docs = prepare_qa_documents('data/home0001qa.json')
chain = setup_rag(docs)

In [38]:
print(chain.invoke("Do i own my 0001 home outright?"))

Yes, when you buy a 0001 home, you own it in the traditional way. You have full ownership and can do whatever you want with it, including buying, selling, or renovating it. You own the title to the home, and it's not a timeshare or rental scheme. It's yours outright.


In [39]:
for chunk in chain.stream("What is Home0001?"):
    print(chunk, end="", flush=True)

Home0001 is a global housing network. Each 0001 home is fully-equipped and furnished, allowing you to move in with just your suitcase. You can also swap cities whenever you like.